In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [2]:
#compute a field which contains all the square roots required
def containing_field(SGA):
    required_square_roots = []
    for partition in Partitions(SGA.group().degree()):
        specht_module = SGA.specht_module(partition)
        rho = specht_module.representation_matrix
        group_size = SGA.group().cardinality()
        P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
        d, L = P.eigenmatrix_left()
        required_square_roots += [specht_module.dimension(),SGA.group().cardinality()] + d.diagonal()
    required_square_roots = flatten([[QQ(q).numerator(),QQ(q).denominator()] if q in QQ else q for q in required_square_roots])
    K = SGA.base_ring()
    for n in set(required_square_roots):
        R = PolynomialRing(K, 'x')
        x = R.gen()
        if (x**2 - n).is_irreducible():
            gen_name = "sqrt"+str(n).replace("/","over")
            K = K.extension(sqrt(n).minpoly(),names=gen_name)
    return K

In [99]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
from sage.matrix.special import diagonal_matrix
from sage.misc.functional import sqrt
def unitary_change_of_basis(SGA,partition,K):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    d, L = P.eigenmatrix_left()
    return L.inverse() * diagonal_matrix([sqrt(K(a)) for a in d.diagonal()]) * L

In [100]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,K,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition,K)
        unitary_factor = specht_module.dimension()/SGA.group().cardinality()
        sqrt_unitary_factor = sqrt(K(unitary_factor))
        return sqrt_unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [101]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [102]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def dft(SGA,unitary=False):
    K = containing_field(SGA)
    fourier_transform = [flatten([hat(delta(g),partition,SGA,K,unitary).list() for partition in Partitions(SGA.group().degree())]) for g in SGA.group()]
    if unitary:
        return matrix(K,fourier_transform).transpose()
    else:
        return matrix(fourier_transform).transpose()

In [103]:
n = 3

In [104]:
SGA = SymmetricGroupAlgebra(QQ,n)

In [105]:
K = containing_field(SGA); K

Number Field in sqrt3 with defining polynomial x^2 - 3 over its base field

In [106]:
G = SGA.group()

In [107]:
SGA_dft = SGA.dft(); SGA_dft

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [108]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]

In [109]:
#compute the unitary DFT of the symmetric group algebra
U_dft = dft(SGA,unitary=True); U_dft.base_ring()

Number Field in sqrt3 with defining polynomial x^2 - 3 over its base field

In [110]:
U_dft*U_dft.conjugate().transpose()

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

In [111]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

True

In [112]:
#QUESTION: what are the eigenvalues?
#for n=3, the minimal polynomial is degree 24 for the eigenvalues. the splitting field is degree 192.
#this means that the Galois group is nonabelian, and since the minimal polynomial has rational coefficients
#the eigenvalues are not roots of unity

In [113]:
U_dft.charpoly()

x^6 + (-1/3*sqrt3 - 1/2)*x^5 + ((-1/6*sqrt2 + 1/6)*sqrt3 - 1/3*sqrt2 - 1/3)*x^4 + ((1/3*sqrt2 + 1/3)*sqrt3 + 1/2*sqrt2)*x^3 + ((-1/6*sqrt2 + 1/6)*sqrt3 - 1/3*sqrt2 - 1/3)*x^2 + (-1/3*sqrt3 - 1/2)*x + 1

In [114]:
#ISSUE: the splitting field appears to be high degree, and the coefficients are large
if len(U_dft.charpoly().factor()) != U_dft.charpoly().degree():
    L.<a> = U_dft.charpoly().splitting_field(map=False); L
else:
    L = K

In [115]:
#getting PARI stack size error
pari.allocatemem(10^10)
pari.stacksize()

PARI stack size set to 10000000000 bytes, maximum size set to 100000006144


10000000000

In [116]:
#form relative exrtension M. attempt to compute Galois group
#for a splitting field L, we should have [L:K] = |Gal(L/K)|
M.<b> = K.extension(U_dft.charpoly()); M.galois_group().order()

KeyboardInterrupt: 

In [117]:
#galois_group() appears to be the absolute version
#NOTE: for a splitting field L, we should have [L:K] = |Gal(L/K)|
#the relative degree is [L:K] = [L:|Q]/[K:\Q]
if L != L.algebraic_closure():
    L_deg = L.absolute_degree(); print(L_deg)
    rel_deg = L_deg/K.absolute_degree(); print(rel_deg)

192
48


In [118]:
#look at all subgroups of order [L:K] in S_d where d is the degree of the polynomial
#for n=3, [L:K] = 48 and it appears there is exactly one subgroup of order 48 up to isomorphism
poly_deg = U_dft.charpoly().degree()
subgroups_order_rel_deg_in_sym = [H for H in SymmetricGroup(poly_deg).subgroups() if H.order() == rel_deg]
all([subgroups_order_rel_deg_in_sym[0].is_isomorphic(H) for H  in subgroups_order_rel_deg_in_sym])

True

In [119]:
#one can factor the polynomial over a splitting field L/K
#but there is no way to express the roots of a quintic in terms of radicals
try:
    eigs = matrix(L,U_dft).eigenvalues(extend=False)
except TypeError:
    print("Cannot express eigenvalues in terms of radicals since polynomial is a quintic or above.")

Cannot express eigenvalues in terms of radicals since polynomial is a quintic or above.


In [120]:
eigs

[-0.9865699009578278? - 0.1633396171296542?*I, -0.9865699009578278? + 0.1633396171296542?*I, -0.5437444089082021? - 0.8392508670124029?*I, -0.5437444089082021? + 0.8392508670124029?*I, 0.9916391752712170? - 0.1290416447020912?*I, 0.9916391752712170? + 0.1290416447020912?*I]

In [121]:
[arg(eig).n(20) for eig in eigs]

[-2.9775, 2.9775, -2.1457, 2.1457, -0.12940, 0.12940]

In [122]:
eigs[0].minpoly()

x^24 + 2*x^23 - 1/2*x^22 - 17/6*x^21 - 215/144*x^20 - 1/6*x^19 + 509/216*x^18 + 11/3*x^17 - 35/1296*x^16 - 139/54*x^15 - 1483/648*x^14 + 7/18*x^13 + 155/54*x^12 + 7/18*x^11 - 1483/648*x^10 - 139/54*x^9 - 35/1296*x^8 + 11/3*x^7 + 509/216*x^6 - 1/6*x^5 - 215/144*x^4 - 17/6*x^3 - 1/2*x^2 + 2*x + 1

In [127]:
U_dft.charpoly().discriminant()

(-213828613/1679616*sqrt2 + 85996015/69984)*sqrt3 + 348251435/62208*sqrt2 - 33598899709/3359232

In [144]:
-sqrt(K(3))

sqrt3

In [141]:
((-213828613/1679616*(sqrt(2)) + 85996015/69984)*(sqrt(3)) + 348251435/62208*(sqrt(2)) - 33598899709/3359232).n()

-268.443660911799

In [ ]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
SGA_dft.base_ring()

In [ ]:
SGA_eigs = SGA_dft.eigenvalues(); SGA_eigs

In [ ]:
[abs(eig) for eig in SGA_eigs]

In [ ]:
SGA_eigs[0].minpoly()

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((SGA_dft*SGA_dft.transpose()).numpy().diagonal()))